# Simulación de un Sistema de Transporte Público Interurbano

### **Objetivo de la Simulación:**

**Simular el Funcionamiento del Sistema de Transporte Público Interurbano**  
El objetivo de esta simulación es modelar el sistema de transporte público entre varias ciudades para observar y analizar su comportamiento bajo diferentes condiciones. La simulación permitirá estudiar cómo varía el tiempo de espera de los pasajeros, la utilización de los vehículos, y la eficiencia general del sistema en función de variables como la demanda de pasajeros, la frecuencia de los servicios, y la capacidad de los vehículos. Además, se busca evaluar el impacto de eventos aleatorios, como retrasos y fluctuaciones en la demanda, para entender mejor el rendimiento del sistema en escenarios reales.

**Descripción del Problema de Colas:**

En esta simulación, el sistema de transporte público se modela como un problema de colas múltiples. Los pasajeros son los "clientes" que llegan a las estaciones (las "colas") y esperan a que un autobús o tren (el "servidor") llegue para transportarlos a su destino.

# Entidades en la Simulación

## 1. **Gasolinera**
La `Gasolinera` maneja el repostaje de vehículos y la carga de gasolina. Se encarga de asegurar que haya suficiente gasolina para los vehículos y gestiona el proceso de repostaje en función de la disponibilidad y la demanda.

## 2. **Passenger**
El `Passenger` simula a los pasajeros que esperan en la estación, abordan un vehículo, viajan hacia su destino y se bajan del vehículo al llegar. Controla la espera en la estación y el proceso de abordaje y descenso del vehículo.

## 3. **Vehicle**
El `Vehicle` representa los vehículos en la simulación. Se encarga de recorrer las rutas, abordar y descender pasajeros, y gestionar el nivel de gasolina. También maneja las estaciones de repostaje y el tiempo de espera en ellas.

## 4. **Setup**
El `Setup` configura la simulación creando vehículos, pasajeros y gasolineras. Se encarga de iniciar la llegada de pasajeros y la asignación de vehículos y estaciones. También organiza la simulación inicial y administra el entorno de la simulación.


# Recursos en la Simulación

## 1. **Resource en Vehicle**
El `Resource` en la clase `Vehicle` gestiona la capacidad de pasajeros en el vehículo. Permite que un número limitado de pasajeros aborden al vehículo al mismo tiempo. La capacidad se define al crear el vehículo y gestiona las solicitudes de abordaje.

## 2. **Container en Gasolinera**
El `Container` en la clase `Gasolinera` se utiliza para almacenar la gasolina disponible en la gasolinera. Gestiona la cantidad de gasolina que se puede almacenar y usar. También permite agregar gasolina al contenedor cuando es necesario.

## 3. **Resource en Gasolinera**
El `Resource` en la clase `Gasolinera` maneja el número de bombas disponibles para repostaje. Limita cuántos vehículos pueden repostar simultáneamente. Esto es crucial para gestionar el tiempo de repostaje y la disponibilidad de las bombas en la gasolinera.



In [949]:
import simpy
import pandas as pd
import random
import numpy as np
import ipdb

# clase Gasolnera

In [951]:


class Gasolinera:
    def __init__(self, env, name, capacity_fuel, capacity_bomb):
        self.env = env
        self.name = name
        self.capacity_fuel = capacity_fuel
        self.capacity_bomb = capacity_bomb
        self.capacidad_gasolina = simpy.Container(env, init=capacity_fuel, capacity=capacity_fuel)
        self.bombas = simpy.Resource(env, capacity=capacity_bomb)
        self.process = env.process(self.run())

        # DataFrame para perfil de la gasolinera
        self.dfP = pd.DataFrame({
            "id": [f"Gasolinera_{name}"],
            "name": [name],
            "capacity_fuel": [capacity_fuel],
            "capacity_bomb": [capacity_bomb],
            "status": ["active"],
            "created_at": [self.env.now]
        })

        # DataFrame para eventos de la gasolinera
        self.dfE = pd.DataFrame(columns=["time", "gasolinera", "event", "detail"])

    def run(self):
        """Proceso que gestiona el repostaje y la carga de gasolina en la gasolinera."""
        while True:
            yield self.env.timeout(1)  # Simular tiempo de operación

    def refuel_vehicle(self, cantidad, name):
        """Proceso para repostar gasolina a un vehículo."""
        with self.bombas.request() as request:
            yield request
            
            # Verificar si hay suficiente gasolina en la gasolinera
            if self.capacidad_gasolina.level >= cantidad:
                cantidad = round(cantidad, 2)
                # Simular el tiempo de repostaje
                time = round(cantidad * 0.1, 2)
                yield self.env.timeout(time)  # Tiempo de repostaje (0.5 min por unidad de gasolina)
                yield self.env.timeout(random.randint(4, 7))
                # Repostar gasolina al vehículo y reducir el nivel de gasolina en la gasolinera
                yield self.capacidad_gasolina.get(cantidad)
                
                print(f'Tiempo {self.env.now}: Repostando {cantidad} l de gasolina en el vehículo {name}.')

                # Registrar evento de repostaje exitoso
                df = pd.DataFrame({
                    "time": [self.env.now],
                    "gasolinera": [self.name],
                    "event": ["refuel"],
                    "cant":[cantidad],
                    "detail": [f"Repostado {cantidad} l en {name}"]
                })
                self.dfE = pd.concat([self.dfE, df])

            else:
                print(f'Tiempo {self.env.now}: No hay suficiente gasolina en la gasolinera para repostar el vehículo {name}.')
                
                # Registrar evento de repostaje fallido
                df = pd.DataFrame({
                    "time": [self.env.now],
                    "gasolinera": [self.name],
                    "event": ["refuel_failed"],
                    "detail": [f"No hay suficiente gasolina para {name}"]
                })
                self.dfE = pd.concat([self.dfE, df])
                
                yield from self.load_gasoline()  # Llamar a la función de instancia

    def load_gasoline(self):
        """Proceso para cargar gasolina en la gasolinera."""
        yield self.env.timeout(20)  # Tiempo para cargar gasolina (5 min en este ejemplo)
        cantidad_cargada = self.capacity_fuel - self.capacidad_gasolina.level
        
        print(f'Tiempo {self.env.now}: Cargada {cantidad_cargada} unidades de gasolina en la gasolinera.')
        yield self.capacidad_gasolina.put(cantidad_cargada)
        print(f'Tiempo {self.env.now}: Cargado {cantidad_cargada} unidades de gasolina en la gasolinera.')

        # Registrar evento de carga de gasolina
        df = pd.DataFrame({
            "time": [self.env.now],
            "gasolinera": [self.name],
            "event": ["carga_gasolina"],
            "cant":[cantidad_cargada],
            "detail": [f"Cargada {cantidad_cargada} l de gasolina"]
        })
        self.dfE = pd.concat([self.dfE, df])


# clase Passenger

In [953]:

class Passenger:
    def __init__(self, env, name, origin, destination, vehicle):
        self.env = env
        self.name = name
        self.origin = origin
        self.destination = destination
        self.vehicle = vehicle
        self.state = False
        self.process = env.process(self.travel())

        # DataFrame para el perfil del pasajero
        self.dfP = pd.DataFrame({
            "name": [name],
            "origin": [origin],
            "destination": [destination],
            "status": ["waiting"],
            "created_at": [self.env.now],
        })

        # DataFrame para eventos del pasajero
        self.dfE = pd.DataFrame(columns=["time", "passenger", "event", "detail"])

    def travel(self):
        
        # Esperar en la estación
        yield from self.wait_in_station()
        # Subirse al vehículo
        yield from self.boardin_vehicle()
        if self.state:
            # Viajar en el vehículo
            yield from self.wait_in_travel()
            # Salir del vehículo
            yield from self.exit_vehicle()

    def wait_in_station(self):
        t = self.env.now
        while self.vehicle.current_route != self.origin:
            yield self.env.timeout(1)

        wait_time = self.env.now - t
        print(f'Tiempo {self.env.now}: {self.name} esperó por {wait_time} min')
         
        # Registrar evento de espera en la estación
        df = pd.DataFrame({
            "time": [self.env.now],
            "passenger": [self.name],
            "vehicle":[self.vehicle.name],
            "event": ["wait_in_station"],
            "cant":[wait_time],
            "detail": [f"Esperó {wait_time} min en la estación {self.origin}"]
        })
        self.dfE = pd.concat([self.dfE, df])

        yield self.env.timeout(1)

    def boardin_vehicle(self):
        self.dfP.loc[self.dfP['name']==self.name,'status']=["board"]
        with self.vehicle.resource.request() as request:
            yield request
            print(f'Tiempo {self.env.now}: {self.name} intenta abordar en {self.origin}')
            self.state = yield self.env.process(self.vehicle.board_passenger(self.name))

            # Registrar evento de abordaje
            if self.state:
                
                df = pd.DataFrame({
                    "time": [self.env.now],
                    "passenger": [self.name],
                    "vehicle":[self.vehicle.name],
                    "event": ["aboard"],
                    "detail": [f"Abordo el vehiculo"]
                })
                self.dfE = pd.concat([self.dfE, df])
            else:
                df = pd.DataFrame({
                    "time": [self.env.now],
                    "passenger": [self.name],
                    "vehicle":[self.vehicle.name],
                    "event": ["aboard failed"],
                    "detail": [f"No pudo abordar el vehiculo"]
                })
                self.dfE = pd.concat([self.dfE, df])
                

    def wait_in_travel(self):
        self.dfP.loc[self.dfP['name']==self.name,'status']=["travel"]
        time_start = self.env.now
        while self.vehicle.current_route != self.destination:
            yield self.env.timeout(random.randint(1, 2))

        travel_time = self.env.now - time_start
        print(f'Tiempo {self.env.now}: {self.name} viajó por {travel_time} min')

        # Registrar evento de viaje
        df = pd.DataFrame({
            "time": [self.env.now],
            "passenger": [self.name],
            "vehicle":[self.vehicle.name],
            "event": ["travel"],
            "cant":[travel_time],
            "detail": [f"Viajó por {travel_time} min hacia {self.destination}"]
        })
        self.dfE = pd.concat([self.dfE, df])

        yield self.env.timeout(1)

    def exit_vehicle(self):
        print(f'Tiempo {self.env.now}: {self.name} llegó a {self.destination}')
        yield from self.vehicle.unboard_passenger()
        self.dfP.loc[self.dfP['name']==self.name,'status']=["arrive"]
        # Registrar evento de salida
        df = pd.DataFrame({
            "time": [self.env.now],
            "passenger": [self.name],
            "vehicle":[self.vehicle.name],
            "event": ["exit_vehicle"],
            "detail": [f"Llegó a {self.destination} y salió del vehículo"]
        })
        self.dfE = pd.concat([self.dfE, df])

        yield self.env.timeout(1)


# Clase Vehiculo

In [955]:

class Vehicle:
    def __init__(self, env, name, route, capacity_passenger, capacity_tank, petrol_station):
        self.env = env
        self.name = name
        self.route = route
        self.capacity_tank = capacity_tank
        self.capacity = simpy.Container(env, init=capacity_passenger, capacity=capacity_passenger)
        self.tank = simpy.Container(env, init=capacity_tank, capacity=capacity_tank)
        self.passenger = 0
        self.current_route = route[0]
        self.resource = simpy.Resource(env, capacity=capacity_passenger)
        self.process = env.process(self.run_route())
        self.petrol_station = petrol_station

        # DataFrame para el perfil del vehículo
        self.dfP = pd.DataFrame({
            "name": [name],
            "route": [route],
            "capacity_passenger": [capacity_passenger],
            "capacity_tank": [capacity_tank],
            "initial_fuel": [capacity_tank],
            "created_at": [self.env.now]
        })

        # DataFrame para eventos del vehículo
        self.dfE = pd.DataFrame(columns=["time", "vehicle", "event", "detail"])

    def run_route(self):
        
        while True:
            # Autobús recorre estaciones
            wait_time = 0
            start_route=self.env.now
            for station in self.route:
                # Vehículo llega a la estación 
                time = random.randint(7, 15)
                # Se utiliza aproximadamente el 5% de tanque por minuto
                gasolina_necesaria = time * (5 / 100)
                
                # Verificar si hay suficiente gasolina
                if self.tank.level >= gasolina_necesaria:
                    # Si es la primera estación, el vehículo no tarda en llegar
                    if self.route.index(station) != 0:
                        yield self.env.timeout(time)
                        print(f'Tiempo {self.env.now}: El {self.name} ha llegado a la estación {station} en {time} min')
                    else:
                        print(f'Tiempo {self.env.now}: El {self.name}  está en la estación {station}')
                        time = 0
                    
                    # Extraer la gasolina
                    yield self.tank.get(gasolina_necesaria)
                    wait_time = self.env.now
                    self.current_route = station

                    # Registrar evento de llegada a la estación
                    df = pd.DataFrame({
                        "time": [self.env.now],
                        "vehicle": [self.name],
                        "tank":[self.tank.level],
                        "station":[self.current_route],
                        "cant passenger":[self.passenger],
                        "event": ["arrive station"],
                        "cant":[time],
                        "detail": [f"Llegó a {station} en {time} min"]
                    })
                    self.dfE = pd.concat([self.dfE, df])
                else:
                    # Si no hay suficiente gasolina, registrar evento de tanque vacío
                    current_tank = self.tank.level
                    new_time = (current_tank * 100) / 5
                    yield self.tank.get(current_tank)
                    print(f'Tanque vacío a {round(time-new_time,5)} min de {station}. El vehículo no puede continuar.')

                    df = pd.DataFrame({
                        "time": [self.env.now],
                        "vehicle": [self.name],
                        "tank":[self.tank.level],
                        "station":[self.current_route],
                        "cant passenger":[self.passenger],
                        "event": ["empty tank"],
                        "cant":[round(time-new_time,5)],
                        "detail": [f"Tanque vacío a {round(time-new_time,5)} min de {station}"]
                    })
                    self.dfE = pd.concat([self.dfE, df])
                    return
                
                # Tiempo de espera antes de salir de la estación
                yield self.env.timeout(random.randint(3, 7))
                
                # Si el nivel de gasolina es bajo y hay estación de carga, recargar combustible
                if station in self.petrol_station.keys() and self.tank.level <= 10:
                    yield from self.chargin_fuel(station)

                print(f'Tiempo {self.env.now}: El {self.name} ha salido de la estación {station} después de {round(self.env.now-wait_time,2)} min')

                # Registrar evento de salida de la estación
                df = pd.DataFrame({
                    "time": [self.env.now],
                    "vehicle": [self.name],
                    "tank":[self.tank.level],
                    "station":[self.current_route],
                    "cant passenger":[self.passenger],
                    "event": ["leave station"],
                    "cant":[round(self.env.now-wait_time,2)],
                    "detail": [f"Salió de {station} después de {round(self.env.now-wait_time,2)} min"]
                })
                self.dfE = pd.concat([self.dfE, df])
                
            # Invertir las rutas
            self.route.reverse()
            df = pd.DataFrame({
                    "time": [self.env.now],
                    "vehicle": [self.name],
                    "tank":[self.tank.level],
                    "station":[self.current_route],
                    "cant passenger":[self.passenger],
                    "event": ["route complete"],
                    "cant":[round(self.env.now-start_route,2)],
                    "detail": [f"completo la ruta"]
                })
            self.dfE = pd.concat([self.dfE, df])

    def chargin_fuel(self, station):
        time_chargin = self.env.now
        cantidad = self.capacity_tank - self.tank.level
        print(f'Tiempo {self.env.now}: Combustible casi agotado en {station}')
        
        yield self.env.timeout(1)
        try:
            yield self.env.process(self.petrol_station[station].refuel_vehicle(cantidad, self.name))
        except simpy.Interrupt as i:
            print(f"{self.name}: message: {i.cause}")
        
        yield self.tank.put(cantidad)
        print(f'Tiempo {self.env.now}: Saliendo de estación de combustible después de {round(self.env.now-time_chargin)} min')

        # Registrar evento de recarga de combustible
        df = pd.DataFrame({
            "time": [self.env.now],
            "vehicle": [self.name],
            "tank":[self.tank.level],
            "station":[self.current_route],
            "cant passenger":[self.passenger],
            "event": ["refueling"],
            "cant":[cantidad],
            "detail": [f"Recargó {cantidad} unidades de combustible en {station}"]
        })
        self.dfE = pd.concat([self.dfE, df])
        
    def board_passenger(self, name):
        if self.capacity.level > 0:
            self.capacity.get(1)
            print(f'Tiempo {self.env.now}: {name} abordó en {self.current_route}')
            yield self.env.timeout(1)
            self.passenger+=1
            # Registrar evento de abordaje de pasajero
            df = pd.DataFrame({
                "time": [self.env.now],
                "vehicle": [self.name],
                "tank":[self.tank.level],
                "station":[self.current_route],
                "cant passenger":[self.passenger],
                "event": ["board passenger"],
                "detail": [f"{name} abordó en {self.current_route}"]
            })
            self.dfE = pd.concat([self.dfE, df])

            return True
        else:
            print(f'Tiempo {self.env.now}: vehículo lleno')
            print(f'Tiempo {self.env.now}: {name} no pudo abordar')

            # Registrar evento de intento fallido de abordaje
            df = pd.DataFrame({
                "time": [self.env.now],
                "vehicle": [self.name],
                "tank":[self.tank.level],
                "station":[self.current_route],
                "cant passenger":[self.passenger],
                "event": ["full capacity"],
                "detail": [f"{name} no pudo abordar, vehículo lleno"]
            })
            self.dfE = pd.concat([self.dfE, df])

            return False

    def unboard_passenger(self):
        self.capacity.put(1)
        yield self.env.timeout(1)
        self.passenger-=1
        # Registrar evento de salida de pasajero
        df = pd.DataFrame({
            "time": [self.env.now],
            "vehicle": [self.name],
            "tank":[self.tank.level],
            "station":[self.current_route],
            "cant passenger":[self.passenger],
            "event": ["exit passenger"],
            "detail": ["Pasajero se bajó del vehículo"]
        })
        self.dfE = pd.concat([self.dfE, df])


# Clase Setup

In [957]:
class Setup:
    def __init__(self, env, num_vehicles, num_passengers, route):
        self.env = env
        self.vehicles = []
        self.passengers = []
        self.route = route
        self.petrol_stations = dict()
        
        for station in np.random.choice(route, 2, replace=False):
            self.petrol_stations[station] = Gasolinera(env, station, random.randint(num_vehicles*30,40*num_vehicles), random.randint(4, 7))

        # Crear vehículos
        for i in range(num_vehicles):
            vehicle = Vehicle(
                env,
                f"Autobus {i+1}",
                route,
                capacity_passenger=random.randint(20, 25),
                capacity_tank=random.randint(20, 30),
                petrol_station=self.petrol_stations
            )
            self.vehicles.append(vehicle)

        # Simular la llegada de pasajeros
        env.process(self.arriving_passengers(num_passengers))

    def arriving_passengers(self, num_passengers):
        for i in range(num_passengers):
            origin, destination = random.sample(self.route, 2)
            passenger = Passenger(
                self.env,
                f"Pasajero {i+1}",
                origin,
                destination,
                random.choice(self.vehicles)
            )
            self.passengers.append(passenger)
            yield self.env.timeout(round(random.random(),2))  # Llegada aleatoria de pasajeros
        #ipdb.set_trace()

    # Método para recolectar todos los perfiles
    def collect_profiles(self):
        #ipdb.set_trace()
        df_vehicles = pd.concat([v.dfP for v in self.vehicles], ignore_index=True)
        df_passengers = pd.concat([p.dfP for p in self.passengers], ignore_index=True)
        df_stations = pd.concat([s.dfP for s in self.petrol_stations.values()], ignore_index=True)
        
        return df_vehicles, df_passengers, df_stations

    # Método para recolectar todos los eventos
    def collect_events(self):
        df_vehicle_events = pd.concat([v.dfE for v in self.vehicles], ignore_index=True)
        df_passenger_events = pd.concat([p.dfE for p in self.passengers], ignore_index=True)
        df_station_events = pd.concat([s.dfE for s in self.petrol_stations.values()], ignore_index=True)
        
        return df_vehicle_events, df_passenger_events, df_station_events


In [ ]:
# Paso 1: Definir rutas y gasolineras
route = ['San Francisco', 'Uvas', 'El Trapiche', 'Altos del Trapiche', 'CU', 'Col. Suyapa']


env = simpy.Environment()


num_vehicles = 10
num_passengers = 2500
setup = Setup(env, num_vehicles, num_passengers, route)



# Paso 4: Ejecutar la simulación
simulation_time = 1000
env.run(until=simulation_time)
vehicle_profiles, passenger_profiles, station_profiles = setup.collect_profiles()
vehicle_events, passenger_events, station_events = setup.collect_events()


# Dataframe de perfiles

In [961]:
station_profiles

,id,name,capacity_fuel,capacity_bomb,status,created_at
0,Gasolinera_Uvas,Uvas,313,6,active,0
1,Gasolinera_Col. Suyapa,Col. Suyapa,302,7,active,0


In [963]:
passenger_profiles

,name,origin,destination,status,created_at
0,Pasajero 1,San Francisco,Altos del Trapiche,arrive,0.00
1,Pasajero 2,CU,San Francisco,arrive,0.86
2,Pasajero 3,El Trapiche,Altos del Trapiche,arrive,1.18
3,Pasajero 4,Altos del Trapiche,CU,arrive,1.36
4,Pasajero 5,El Trapiche,Uvas,arrive,2.05
...,...,...,...,...,...
2005,Pasajero 2006,San Francisco,Altos del Trapiche,waiting,997.14
2006,Pasajero 2007,Altos del Trapiche,CU,waiting,997.53
2007,Pasajero 2008,San Francisco,El Trapiche,waiting,997.95
2008,Pasajero 2009,Altos del Trapiche,CU,waiting,998.73


In [965]:
vehicle_profiles

,name,route,capacity_passenger,capacity_tank,initial_fuel,created_at
0,Autobus 1,"[Col. Suyapa, CU, Altos del Trapiche, El Trapi...",20,26,26,0
1,Autobus 2,"[Col. Suyapa, CU, Altos del Trapiche, El Trapi...",25,29,29,0
2,Autobus 3,"[Col. Suyapa, CU, Altos del Trapiche, El Trapi...",20,26,26,0
3,Autobus 4,"[Col. Suyapa, CU, Altos del Trapiche, El Trapi...",24,23,23,0
4,Autobus 5,"[Col. Suyapa, CU, Altos del Trapiche, El Trapi...",20,20,20,0
5,Autobus 6,"[Col. Suyapa, CU, Altos del Trapiche, El Trapi...",20,21,21,0
6,Autobus 7,"[Col. Suyapa, CU, Altos del Trapiche, El Trapi...",25,28,28,0
7,Autobus 8,"[Col. Suyapa, CU, Altos del Trapiche, El Trapi...",20,24,24,0
8,Autobus 9,"[Col. Suyapa, CU, Altos del Trapiche, El Trapi...",24,26,26,0
9,Autobus 10,"[Col. Suyapa, CU, Altos del Trapiche, El Trapi...",20,22,22,0


# Dataframes de eventos 

In [967]:
station_events

,time,gasolinera,event,detail,cant
0,295.15,Uvas,refuel,Repostado 11.45 l en Autobus 6,11.45
1,351.37,Uvas,refuel,Repostado 13.65 l en Autobus 4,13.65
2,436.70,Uvas,refuel,Repostado 16.95 l en Autobus 9,16.95
3,705.74,Uvas,refuel,Repostado 13.65 l en Autobus 4,13.65
4,897.46,Uvas,refuel,Repostado 16.6 l en Autobus 1,16.60
5,944.67,Uvas,refuel,Repostado 16.65 l en Autobus 3,16.65
6,341.23,Col. Suyapa,refuel,Repostado 12.3 l en Autobus 5,12.30
7,346.32,Col. Suyapa,refuel,Repostado 13.15 l en Autobus 10,13.15
8,446.69,Col. Suyapa,refuel,Repostado 16.9 l en Autobus 8,16.90
9,454.80,Col. Suyapa,refuel,Repostado 18.0 l en Autobus 1,18.00


In [969]:
passenger_events

,time,passenger,event,detail,vehicle,cant
0,0,Pasajero 1,wait_in_station,Esperó 0 min en la estación San Francisco,Autobus 8,0.0
1,2,Pasajero 1,aboard,Abordo el vehiculo,Autobus 8,NaN
2,40,Pasajero 1,travel,Viajó por 38 min hacia Altos del Trapiche,Autobus 8,38.0
3,42,Pasajero 1,exit_vehicle,Llegó a Altos del Trapiche y salió del vehículo,Autobus 8,NaN
4,60.86,Pasajero 2,wait_in_station,Esperó 60.0 min en la estación CU,Autobus 2,60.0
...,...,...,...,...,...,...
6785,998.02,Pasajero 1984,aboard,Abordo el vehiculo,Autobus 4,NaN
6786,991.0,Pasajero 1994,wait_in_station,Esperó 0.0 min en la estación Col. Suyapa,Autobus 4,0.0
6787,993.0,Pasajero 1994,aboard,Abordo el vehiculo,Autobus 4,NaN
6788,992.57,Pasajero 1997,wait_in_station,Esperó 0.0 min en la estación San Francisco,Autobus 1,0.0


In [971]:
vehicle_events

,time,vehicle,event,detail,tank,station,cant passenger,cant
0,0,Autobus 1,arrive station,Llegó a San Francisco en 0 min,25.60,San Francisco,0.0,0.0
1,3,Autobus 1,leave station,Salió de San Francisco después de 3 min,25.60,San Francisco,0.0,3.0
2,12,Autobus 1,arrive station,Llegó a Uvas en 9 min,25.15,Uvas,0.0,9.0
3,14.49,Autobus 1,board passenger,Pasajero 10 abordó en Uvas,25.15,Uvas,1.0,NaN
4,16,Autobus 1,leave station,Salió de Uvas después de 4 min,25.15,Uvas,1.0,4.0
...,...,...,...,...,...,...,...,...
4908,978.61,Autobus 10,arrive station,Llegó a Uvas en 11 min,10.25,Uvas,11.0,11.0
4909,980.87,Autobus 10,board passenger,Pasajero 1962 abordó en Uvas,10.25,Uvas,12.0,NaN
4910,980.89,Autobus 10,board passenger,Pasajero 1950 abordó en Uvas,10.25,Uvas,13.0,NaN
4911,984.46,Autobus 10,board passenger,Pasajero 1978 abordó en Uvas,10.25,Uvas,14.0,NaN


# Análisis y Preguntas sobre la Simulación (20 puntos)

### 1. ¿Cuál es el tiempo promedio que un pasajero pasa esperando en la estación antes de abordar un vehículo?

In [973]:
df_new=passenger_events.loc[passenger_events["event"]=="wait_in_station"]
df_new

,time,passenger,event,detail,vehicle,cant
0,0,Pasajero 1,wait_in_station,Esperó 0 min en la estación San Francisco,Autobus 8,0.0
4,60.86,Pasajero 2,wait_in_station,Esperó 60.0 min en la estación CU,Autobus 2,60.0
8,33.18,Pasajero 3,wait_in_station,Esperó 32.0 min en la estación El Trapiche,Autobus 2,32.0
12,39.36,Pasajero 4,wait_in_station,Esperó 38.0 min en la estación Altos del Trapiche,Autobus 8,38.0
16,35.05,Pasajero 5,wait_in_station,Esperó 33.0 min en la estación El Trapiche,Autobus 5,33.0
...,...,...,...,...,...,...
6780,979.41,Pasajero 1974,wait_in_station,Esperó 0.0 min en la estación El Trapiche,Autobus 2,0.0
6782,982.46,Pasajero 1978,wait_in_station,Esperó 0.0 min en la estación Uvas,Autobus 10,0.0
6784,996.02,Pasajero 1984,wait_in_station,Esperó 10.0 min en la estación CU,Autobus 4,10.0
6786,991.0,Pasajero 1994,wait_in_station,Esperó 0.0 min en la estación Col. Suyapa,Autobus 4,0.0


In [975]:
prom_wait_time=df_new["time"].mean()
prom_wait_time

522.7554491662187

### 2.¿Cuál es el tiempo promedio que un vehículo tarda en recorrer una ruta completa?

In [977]:
df_new=vehicle_events.loc[ vehicle_events["event"]=="route complete"]
df_new["cant"].mean()

86.34621621621622

### 3.¿Cuántos pasajeros son atendidos por cada vehículo durante la simulación?

In [979]:
df_new3=passenger_events.loc[ passenger_events["event"]=="aboard"]
df_new3=df_new3.groupby('vehicle')['passenger'].count()
df_new3

vehicle
Autobus 1     179
Autobus 10    166
Autobus 2     133
Autobus 3     178
Autobus 4     188
Autobus 5     146
Autobus 6     190
Autobus 7     187
Autobus 8     149
Autobus 9     173
Name: passenger, dtype: int64

### 4. ¿Cuál es el tiempo promedio de espera en la estación para repostaje de combustible?

In [981]:
df_new4=vehicle_events.loc[vehicle_events["event"]=="refueling"]
df_new4["cant"].mean()

15.342105263157892

### 5. ¿Cuántas veces se agota el combustible en los vehículos durante la simulación?


In [983]:
df_new5=vehicle_events.loc[vehicle_events["event"]=="refueling"].groupby('vehicle')["vehicle"].count()
df_new5

vehicle
Autobus 1     2
Autobus 10    2
Autobus 2     1
Autobus 3     2
Autobus 4     2
Autobus 5     3
Autobus 6     2
Autobus 7     1
Autobus 8     2
Autobus 9     2
Name: vehicle, dtype: int64

### 6. ¿Qué porcentaje de las veces que un pasajero intenta abordar un vehículo tiene éxito?

In [985]:
df_new6= passenger_events.loc[ (passenger_events['event']=="aboard")| (passenger_events['event']=="aboard failed")]
passenger_events.loc[ (passenger_events['event']=="aboard")].shape[0]/df_new6.shape[0]

0.9085529854760624

### 7. ¿Cuál es el nivel promedio de gasolina en los vehículos al finalizar la simulación?

In [987]:
df_new7 =vehicle_events.sort_values(by='time').groupby('vehicle').tail(1)
df_new7

,time,vehicle,event,detail,tank,station,cant passenger,cant
4912,985.61,Autobus 10,leave station,Salió de Uvas después de 7.0 min,10.25,Uvas,14.0,7.0
1483,988.67,Autobus 3,leave station,Salió de CU después de 6.0 min,24.65,CU,14.0,6.0
4416,989.74,Autobus 9,exit passenger,Pasajero se bajó del vehículo,25.65,Uvas,18.0,NaN
2446,993.28,Autobus 5,leave station,Salió de El Trapiche después de 4.0 min,15.40,El Trapiche,20.0,4.0
502,994.57,Autobus 1,board passenger,Pasajero 1997 abordó en San Francisco,21.65,San Francisco,14.0,NaN
2964,995.38,Autobus 6,board passenger,Pasajero 1898 abordó en Uvas,6.65,Uvas,18.0,NaN
3488,996.99,Autobus 7,leave station,Salió de CU después de 6.0 min,9.80,CU,13.0,6.0
3928,997.3,Autobus 8,leave station,Salió de Altos del Trapiche después de 5.0 min,19.60,Altos del Trapiche,17.0,5.0
1996,999.67,Autobus 4,exit passenger,Pasajero se bajó del vehículo,11.55,CU,12.0,NaN
960,999.91,Autobus 2,route complete,completo la ruta,10.40,San Francisco,13.0,85.0


In [989]:
prom=df_new7["tank"].mean()
prom

15.559999999999999

### 8. ¿Cuál es la cantidad total de gasolina cargada en cada gasolinera?

In [991]:
df_new8=station_events.loc[station_events["event"]=="refuel"].groupby('gasolinera').agg({
    "cant":["sum"]
})

df_new8

,cant
,sum
gasolinera,
Col. Suyapa,202.55
Uvas,88.95


### 9. ¿Cuántos pasajeros abordaron un vehículo en cada estación?

In [999]:
df_new9=vehicle_events.loc[vehicle_events["event"]=="board passenger"].groupby("station")["station"].count()
df_new9

station
Altos del Trapiche    268
CU                    281
Col. Suyapa           298
El Trapiche           294
San Francisco         254
Uvas                  294
Name: station, dtype: int64

### 10. ¿Cuántos pasajeros abandonan la simulación sin haber abordado un vehículo?

In [1001]:
df_new10=passenger_profiles.loc[passenger_profiles["status"]=="waiting"]
df_new10
df_new10.shape[0]

151

# Conclusiones

La simulacion se puede hacer mucho mas realista tomando muchos aspectos en consideracino como : 
  - `Trafico`
  - `Semaforos`
  - `Desperfectos mecanicos`
  - `Accidentes`

sin embargo no deja de ser un buen ejemplo para mostrar como se hacen las simulaciones
 